In [1]:
import numpy as np

In [2]:
class Lab1:
    def __init__(self, n):
        self.n = n


    def get_X(self, begin, end, size) : return np.random.normal(begin, end, size = size)


    def get_a(self, X) : return np.divide(np.sum(X), len(X))


    def get_sigma(self, X) : return np.sqrt(np.divide(np.sum((X - self.get_a(X)) ** 2), len(X)))


    def solve_A(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i); a = self.get_a(X); sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i - 1); r = a + z * sigma / np.sqrt(n_i - 1)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_B(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i); a = self.get_a(X); sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i) ;r = a + z * sigma / np.sqrt(n_i)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_C(self, z, variety):
        for n_i, z_i in zip(self.n, z):
            X = self.get_X(0, variety, n_i); sigma = self.get_sigma(X)
            l = n_i * sigma / z_i[1]; r = n_i * sigma / z_i[0]
            print(f'n = {n_i}: ',\
                  f'\n\t{variety} попал в интервал ({l}, {r})\n' if variety > l and variety < r \
                  else f'\n\t{variety} попал в интервал ({l}, {r})\n')

In [3]:
L1 = Lab1(np.array([100, 10000, 1000000]))

In [4]:
L1.solve_A(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.15931700088738326, 0.3998725209212946)

n = 10000:  
	0.0 попал в интервал (-0.020775566229277006, 0.030576508702203787)

n = 1000000:  
	0.0 попал в интервал (-0.0037500933890752862, 0.001392870726203637)



In [5]:
L1.solve_B(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.22830235077807598, 0.2426397780553931)

n = 10000:  
	0.0 попал в интервал (-0.016559893077170935, 0.03462667623818638)

n = 1000000:  
	0.0 попал в интервал (-0.004331594031255234, 0.0008168823945772469)



In [6]:
z = [(66.5101053, 138.9867835), (9638.498197, 10367.01484), (996359.9858, 1003645.527)]

L1.solve_C(z, 1.)

n = 100:  
	1.0 попал в интервал (0.7232566300147248, 1.5113960833076017)

n = 10000:  
	1.0 попал в интервал (0.9612034040123321, 1.0338550415204648)

n = 1000000:  
	1.0 попал в интервал (0.9968544955930254, 1.0041436527265455)



In [7]:
from numba import jit
# оформить все это в виде класса

In [8]:
@jit(nopython = True, parallel = True)
def get_sequence(size) : return np.random.uniform(0.0, 1.0, size = int(size))

In [9]:
@jit(nopython = True, parallel = True)
def get_ksi(w, alpha) : return ((-np.log(w)) ** 0.25) / alpha

In [10]:
@jit(nopython = True, parallel = True)
def get_etha(w) : return np.sqrt(-np.log(w))

In [11]:
@jit(nopython = True, parallel = True)
def get_count_nonzero(arr) : return np.count_nonzero(arr == 1)

In [12]:
@jit(nopython = True, parallel = True)
def task_B(size, alpha):
    k = get_ksi(get_sequence(size), alpha); t = get_etha(get_sequence(size))
#     print(k[:50])
#     print(t[:50])
    return k < t

# дописать остальные таски

In [13]:
@jit(nopython = True, parallel = True)
def calc_result(n_0, alpha):
    z2 = 2.575 ** 2; e2 = 0.01 ** 2; n = n_0
    sequence = task_B(n_0, alpha)
    summ = sequence.sum(); summ_squared = (sequence ** 2).sum()
#     print(summ, summ_squared)
    assert summ != 0.0, 'Нужно n_0 побольше' 
    Q = summ / n
    sigma2 = (summ_squared - n * Q * Q) / (n - 1)
    criteria = (z2 * sigma2) / (e2 * Q * Q)
#     start = time.time()
    step = 5000
    while n < criteria:
        n += step
#         if n % 1000 == 0:
#             print(n, criteria)
        q = task_B(step, alpha)
        summ += q.sum(); summ_squared += (q ** 2).sum()
        Q = summ / n;
        sigma2 = (summ_squared - n * Q * Q) / (n - 1)
        criteria = (z2 * sigma2) / (e2 * Q * Q)
        
#     print('after first loop', n, criteria)
    while n > np.floor(criteria) + 1.0:
#         print(n, criteria)
        n -= 1
        q = task_B(1, alpha)
        summ -= q.sum(); summ_squared -= (q ** 2).sum()
        Q = summ / n
        sigma2 = (summ_squared - n * Q * Q) / (n - 1)
        criteria = (z2 * sigma2) / (e2 * Q * Q)
#     print(n, criteria)
    return alpha, n, Q#, time.time() - start

In [19]:
%%time
result = calc_result(1000000, 0.1)
names = ['Alpha', 'n*', 'Q']
for name, value in zip(names, result):
    print(f'{name} = {value}')

Alpha = 0.1
n* = 333704766
Q = 0.00019865763619330507
CPU times: user 2min 23s, sys: 42.5 s, total: 3min 5s
Wall time: 53.5 s
